In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [ ]:
import random
import numpy as np
import skimpy
import ipywebrtc
from skimpy import functional as F
from skimpy_3d import csg, isosurface, voxels
from skimpy_blox import minecraft, colors
import tools.notebooks as nb

In [ ]:
%%time
level = minecraft.SkimpyMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

In [ ]:
%%time
t = level.megatensor().min(1).to(float)

In [ ]:
t.shape

In [ ]:
%%time
mesh = isosurface.marching_cubes(t, 0.1)

In [ ]:
nb.render_mesh(mesh, up_vector=(0, 0, 1))